In [1]:


import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_e481334d51e84673ab601b9d741817dd = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='7IaSoJvfoNiVUd8AgMF-ad3qNZVjwuvTIpMvynHX931k',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_e481334d51e84673ab601b9d741817dd.get_object(Bucket='ml1-donotdelete-pr-wnk3amdcl8llcz',Key='life_prediction.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.head()



,Country,Status,Year,Schooling,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,...,Polio,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population,thinness 1-19 years,thinness 5-9 years,Income composition of resources,Life expectancy
0,Afghanistan,Developing,2015,10.1,263.0,62,0.01,71.279624,65.0,1154,...,6.0,8.16,65.0,0.1,584.259210,33736494.0,17.2,17.3,0.479,65.0
1,Afghanistan,Developing,2014,10.0,271.0,64,0.01,73.523582,62.0,492,...,58.0,8.18,62.0,0.1,612.696514,327582.0,17.5,17.5,0.476,59.9
2,Afghanistan,Developing,2013,9.9,268.0,66,0.01,73.219243,64.0,430,...,62.0,8.13,64.0,0.1,631.744976,31731688.0,17.7,17.7,0.470,59.9
3,Afghanistan,Developing,2012,9.8,272.0,69,0.01,78.184215,67.0,2787,...,67.0,8.52,67.0,0.1,669.959000,3696958.0,17.9,18.0,0.463,59.5
4,Afghanistan,Developing,2011,9.5,275.0,71,0.01,7.097109,68.0,3013,...,68.0,7.87,68.0,0.1,63.537231,2978599.0,18.2,18.2,0.454,59.2


In [2]:


#df_data_1 = df_data_1.replace(0, null)
# fill missing values with mean column values
df_data_1.fillna(df_data_1.mean(), inplace=True)
# count the number of NaN values in each column
print(df_data_1.isnull().sum())


x=df_data_1.iloc[:,1:-1].values
y=df_data_1.iloc[:,-1].values
print(x)



Country                            0
Status                             0
Year                               0
Schooling                          0
Adult Mortality                    0
infant deaths                      0
Alcohol                            0
percentage expenditure             0
Hepatitis B                        0
Measles                            0
 BMI                               0
under-five deaths                  0
Polio                              0
Total expenditure                  0
Diphtheria                         0
 HIV/AIDS                          0
GDP                                0
Population                         0
 thinness  1-19 years              0
 thinness 5-9 years                0
Income composition of resources    0
Life expectancy                    0
dtype: int64
[['Developing' 2015 10.1 ... 17.2 17.3 0.479]
 ['Developing' 2014 10.0 ... 17.5 17.5 0.47600000000000003]
 ['Developing' 2013 9.9 ... 17.7 17.7 0.47]
 ...
 ['Developing' 200

In [3]:


from sklearn.preprocessing import LabelEncoder,OneHotEncoder
labelencoder_x = LabelEncoder()
x[:,0]=labelencoder_x.fit_transform(x[:,0])
#x[:,1]=labelencoder_x.fit_transform(x[:,1])
print(x)



[[1 2015 10.1 ... 17.2 17.3 0.479]
 [1 2014 10.0 ... 17.5 17.5 0.47600000000000003]
 [1 2013 9.9 ... 17.7 17.7 0.47]
 ...
 [1 2002 10.0 ... 1.2 1.3 0.42700000000000005]
 [1 2001 9.8 ... 1.6 1.7 0.42700000000000005]
 [1 2000 9.8 ... 11.0 11.2 0.434]]


In [4]:
from sklearn.preprocessing import Imputer
imputer = Imputer(missing_values='NaN',strategy='mean',axis=0)

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/utils/deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [5]:
imputer = imputer.fit(x[:,:])
x[:,:]=imputer.transform(x[:,:])
print(x)

[[1.0 2015.0 10.1 ... 17.2 17.3 0.479]
 [1.0 2014.0 10.0 ... 17.5 17.5 0.47600000000000003]
 [1.0 2013.0 9.9 ... 17.7 17.7 0.47]
 ...
 [1.0 2002.0 10.0 ... 1.2 1.3 0.42700000000000005]
 [1.0 2001.0 9.8 ... 1.6 1.7 0.42700000000000005]
 [1.0 2000.0 9.8 ... 11.0 11.2 0.434]]


In [6]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=1/3,random_state = 0)

In [7]:
from sklearn.ensemble import RandomForestRegressor
reg = RandomForestRegressor(n_estimators=10,random_state=0)
reg.fit(x,y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
           oob_score=False, random_state=0, verbose=0, warm_start=False)

In [8]:
y_pred = reg.predict(x_test)

accu = reg.score(x_test,y_test)
print("accuracy is",accu)

accuracy is 0.9925504422226907


In [16]:


from watson_machine_learning_client import WatsonMachineLearningAPIClient
wml_credentials={
  "apikey": "fNyrDbklMnG9B9QSuJm8k_BrJ5N79n1EjyTpNt54veHp",
  "iam_apikey_description": "Auto-generated for key b79af1ed-31f5-49a7-9469-131f1f8e5e84",
  "iam_apikey_name": "wdp-writer",
  "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Writer",
  "iam_serviceid_crn": "crn:v1:bluemix:public:iam-identity::a/4093fb5649df43d8b29f8d9d34877965::serviceid:ServiceId-50a9bafe-a009-4f80-a381-19b4ec7b6e6f",
  "instance_id": "989184fd-14f3-4a3a-a2a9-01bfd1e3fdaf",
  "url": "https://eu-gb.ml.cloud.ibm.com"
}


In [17]:

client = WatsonMachineLearningAPIClient(wml_credentials)



In [18]:


metadata={
    client.repository.ModelMetaNames.DESCRIPTION:'life prediction',

    client.repository.ModelMetaNames.AUTHOR_NAME:'Vedanti',
    
    client.repository.ModelMetaNames.AUTHOR_EMAIL:'vedanti.vyawahare17@vit.edu',

    client.repository.ModelMetaNames.NAME:"Multilinear"

}



In [19]:


model_details=client.repository.store_model(reg,meta_props=metadata)
print(model_details)



{'metadata': {'guid': '32103dff-90d7-446f-8f62-2ac2138e2298', 'url': 'https://eu-gb.ml.cloud.ibm.com/v3/wml_instances/989184fd-14f3-4a3a-a2a9-01bfd1e3fdaf/published_models/32103dff-90d7-446f-8f62-2ac2138e2298', 'created_at': '2020-07-07T08:56:45.606Z', 'modified_at': '2020-07-07T08:56:45.663Z'}, 'entity': {'runtime_environment': 'python-3.6', 'learning_configuration_url': 'https://eu-gb.ml.cloud.ibm.com/v3/wml_instances/989184fd-14f3-4a3a-a2a9-01bfd1e3fdaf/published_models/32103dff-90d7-446f-8f62-2ac2138e2298/learning_configuration', 'author': {'name': 'Vedanti'}, 'name': 'Multilinear', 'description': 'life prediction', 'learning_iterations_url': 'https://eu-gb.ml.cloud.ibm.com/v3/wml_instances/989184fd-14f3-4a3a-a2a9-01bfd1e3fdaf/published_models/32103dff-90d7-446f-8f62-2ac2138e2298/learning_iterations', 'feedback_url': 'https://eu-gb.ml.cloud.ibm.com/v3/wml_instances/989184fd-14f3-4a3a-a2a9-01bfd1e3fdaf/published_models/32103dff-90d7-446f-8f62-2ac2138e2298/feedback', 'latest_version'

In [20]:
published_model_uid=client.repository.get_model_uid(model_details)
published_model_uid

'32103dff-90d7-446f-8f62-2ac2138e2298'

In [21]:
created_deployment=client.deployments.create(published_model_uid,name="prediction")
scoring_endpoint = client.deployments.get_scoring_url(created_deployment)
client.deployments.list()



#######################################################################################

Synchronous deployment creation for uid: '32103dff-90d7-446f-8f62-2ac2138e2298' started

#######################################################################################


INITIALIZING
DEPLOY_SUCCESS


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='8a942dfb-1456-4681-8510-889dbbc9a438'
------------------------------------------------------------------------------------------------


------------------------------------  ----------  ------  --------------  ------------------------  -----------------  -------------
GUID                                  NAME        TYPE    STATE           CREATED                   FRAMEWORK          ARTIFACT TYPE
8a942dfb-1456-4681-8510-889dbbc9a438  prediction  online  DEPLOY_SUCCESS  2020-07-07T08:56:57.224Z  scikit-learn-0.20  model
----------------

In [22]:
scoring_endpoint

'https://eu-gb.ml.cloud.ibm.com/v3/wml_instances/989184fd-14f3-4a3a-a2a9-01bfd1e3fdaf/deployments/8a942dfb-1456-4681-8510-889dbbc9a438/online'